Импортируем библиотеки.

In [ ]:
import numpy as np
import pandas as pd
import sys
import warnings

warnings.filterwarnings('ignore')

In [ ]:
main_df = pd.read_excel('Задание 5.0 2 версия.xlsx')
main_df
main_df[main_df['Профессия'] == 'Уборщик'] 

In [ ]:
# main_df['Оборудование'].value_counts()
main_df['Профессия'].values

#### подумай: а оно надо?


Преобразование месяцев в числа. Заметим что все месяцы даны в формате timestamp, их диапазон от января по декабрь 2009 года (то есть данные приведены за один год, что сильно упрощает задачу)

In [ ]:
main_df['Месяц'].max(), main_df['Месяц'].min()

С целью небольшой экономии памяти заменим месяцы в формате timesamp на числа формата int в диапазоне от 1 до 12.

In [ ]:
for j in range(len(main_df['Месяц'])):
    main_df['Месяц'][j] = main_df['Месяц'][j].month

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# sns.barplot(y = "Профессия", x = "Доход",data = main_df)

In [ ]:
# sns.barplot(y = "Оборудование", x = "Вредность (балл.)",data = main_df)

In [ ]:
# sns.barplot(y = "Оборудование", x = "Доход",data = main_df)

In [ ]:
pd.unique(main_df['Профессия'].values)

Здесь мы пытаемся объединить профессии по группам.

In [ ]:
# Попробуем объединить группы по признаку

prof_group = (
    {
        'Оператор поста управления', 
        'Подручный вальцовщика',       # группа 1
        'Резчик горячего металла',
        'Огнеупорщик'
    },
    
    {
        'Вальцовщик',
        'Нагревальщик металла',        # группа 2
        'Контролер качества готовой продукции'
    },
    
    {
        'Уборщик'                      # группа 3
    },
    
    {
         'Посадчик металла'            # группа 4
    }
)


for i, group in enumerate(prof_group):
    main_df['Профессия'][main_df['Профессия'].isin(group)] = f'group_{i+1}'

In [ ]:
equipment_group = (
    {
        'Пилигримовые станы', 
        'Калибровочный стан',       # группа 1
        'Калибровочный стан, ГРП',
    },
    
    {
        'Нагревательные печи, нагрев',
        'Нагревательные печи',        # группа 2
        'Нагревательные печи, кантовка'
    },
    
    {
        'Контрольно-измерительная аппаратура',
        'Прошивной стан'# группа 3
    },
    
    {
         'Вспомогательные помещения'            # группа 4
    }
)

# pd.unique(main_df['Оборудование'].values)

for i, group in enumerate(equipment_group):
    main_df['Оборудование'][main_df['Оборудование'].isin(group)] = f'equipment_{i+1}'

In [ ]:
main_df

Уберем из датафрейма месяцы и имена. Грубо приблизим, что оклад платится не подневно, а помесячно (то есть оклад не зависит от количества дней в месяце).

In [ ]:
anon_df = main_df.drop(columns = ['FIO', 'Месяц', 'Оборудование'])

In [ ]:
if 'Профессия' in  anon_df.columns or 'Оборудование' in anon_df.columns:
#     anon_df = pd.get_dummies(anon_df, columns = ['Профессия', 'Оборудование'])
     anon_df = pd.get_dummies(anon_df, columns = ['Профессия'])
anon_df

Обучим в первом приближении

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

y_anon_df = anon_df['Доход']
X_anon_df = anon_df.drop(columns = ['Доход'])
X_ = X_anon_df
# X_= scaler.fit_transform(X_anon_df)

regr = LinearRegression(positive = True)
regr.fit(X_, y_anon_df)

regr.intercept_, regr.coef_

# pd.DataFrame([regr.feature_names_in_, regr.coef_]).T

In [ ]:
df